---
title: "sound"
execute:
  # echo: false
  freeze: auto  # re-render only when source changes
format:
  html:
    code-fold: true
    code-summary: "Show the code"
---

Sound source: [University of Iowa Electronic Music Studios](https://theremin.music.uiowa.edu/index.html)

Listen to these three notes on the piano

![](keyboard_3notes.png)

::: {.column-margin}
Image modified from: [piano-music-theory.com](https://piano-music-theory.com/2016/05/28/the-piano-keyboard/)
:::


A3
<audio controls="controls"><source src="Piano.mf.A3.aiff" type="audio/aiff" /></audio>  
A4 
<audio controls="controls"><source src="Piano.mf.A4.aiff" type="audio/aiff" /></audio>  
B4
<audio controls="controls"><source src="Piano.mf.B4.aiff" type="audio/aiff" /></audio>

In [1]:
#| code-summary: "import stuff"
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import aifc
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()  # datetime converter for a matplotlib
import seaborn as sns
sns.set(style="ticks", font_scale=1.5)

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.templates.default = "presentation"
import math
import scipy

# %matplotlib widget

In [2]:
#| code-summary: "define useful functions"
def open_aiff(filename):
    data = aifc.open(filename)
    Nframes = data.getnframes()
    str_signal = data.readframes(Nframes)
    # d = np.fromstring(str_signal, numpy.short).byteswap()
    d = np.frombuffer(str_signal, dtype=np.int16).byteswap()
    # if there are two channels, take only one of them
    if data.getnchannels() == 2:
        d = d[::2]
    N = len(d)
    dt = 1.0 / data.getframerate()
    time = np.arange(N) * dt
    return time, d

def normalize(signal):
    return (signal - signal.mean()) / signal.std()

def fft_abs(signal, time, max_freq=None):
    dt = time[1] - time[0]
    N = len(time)
    fft = scipy.fft.fft(signal) / N
    k = scipy.fft.fftfreq(N, dt)
    fft_abs = np.abs(fft)
    if max_freq == None:
        max_freq = k.max()
    dk = 1 / (timeA4.max() - timeA4.min())
    n = int(max_freq/dk)
    return k[:n], fft_abs[:n]
    

In [3]:
#| code-summary: "load audio files"
tmin = 2.00
tmax = 2.20

timeA4, pianoA4 = open_aiff('Piano.mf.A4.aiff')
indices = np.where((timeA4 > tmin) & (timeA4 < tmax))
timeA4 = timeA4[indices]
pianoA4 = normalize(pianoA4[indices])

timeA3, pianoA3 = open_aiff('Piano.mf.A3.aiff')
indices = np.where((timeA3 > tmin) & (timeA3 < tmax))
timeA3 = timeA3[indices]
pianoA3 = normalize(pianoA3[indices])

timeB4, pianoB4 = open_aiff('Piano.mf.B4.aiff')
indices = np.where((timeB4 > tmin) & (timeB4 < tmax))
timeB4 = timeB4[indices]
pianoB4 = normalize(pianoB4[indices])

In [4]:
#| code-summary: "plot time series for 3 notes"
# blue, orange, green, red, purple, brown, pink, gray, olive, cyan = px.colors.qualitative.D3[:]

color_A3 = "#e7c535"
color_A4 = "#ff00c3"
color_B4 = "#327fce"

fig = make_subplots()

fig.add_trace(
    go.Scatter(x=list(timeA3),
               y=list(pianoA3),
               name='piano A3',
               line=dict(color=color_A3),
               visible='legendonly'),
)

fig.add_trace(
    go.Scatter(x=list(timeA4),
               y=list(pianoA4),
               name='piano A4',
               line=dict(color=color_A4),
               ),
)

fig.add_trace(
    go.Scatter(x=list(timeB4),
               y=list(pianoB4),
               name='piano B4',
               line=dict(color=color_B4),
               visible='legendonly'),
)

# Add range slider
fig.update_layout(
    title='3 notes on the piano',
    xaxis=dict(
        rangeslider={"visible":True},
        type="linear"
    ),
    legend={"orientation":"h",           # Horizontal legend
            "yanchor":"top",             # Anchor legend to the top
            "y":1.1,                    # Adjust vertical position
            "xanchor":"center",           # Anchor legend to the right
            "x":0.5,                       # Adjust horizontal position
           },
)

# Set y-axes titles
fig.update_yaxes(
    title_text="signal",
    range=[-3,3])
fig.update_xaxes(
    title_text="time (s)",)

In [5]:
#| code-summary: "calculate fourier transform"
kA4, abs_fft_A4 = fft_abs(pianoA4, timeA4, max_freq=4e3)
kA3, abs_fft_A3 = fft_abs(pianoA3, timeA3, max_freq=4e3)
kB4, abs_fft_B4 = fft_abs(pianoB4, timeB4, max_freq=4e3)

In [6]:
#| code-summary: "plot power spectrum for 3 notes"

fig = make_subplots()

fig.add_trace(
    go.Scatter(x=list(kA3),
               y=list(abs_fft_A3),
               name='piano A3',
               line=dict(color=color_A3),
               visible='legendonly'),
)

fig.add_trace(
    go.Scatter(x=list(kA4),
               y=list(abs_fft_A4),
               name='piano A4',
               line=dict(color=color_A4),),
)

fig.add_trace(
    go.Scatter(x=list(kB4),
               y=list(abs_fft_B4),
               name='piano B4',
               line=dict(color=color_B4),
               visible='legendonly'),
)

# Add range slider
fig.update_layout(
    title='3 notes on the piano',
    legend={"orientation":"h",           # Horizontal legend
            "yanchor":"top",             # Anchor legend to the top
            "y":1.1,                    # Adjust vertical position
            "xanchor":"center",           # Anchor legend to the right
            "x":0.5,                       # Adjust horizontal position
           },
)

# Set y-axes titles
fig.update_yaxes(
    title_text="abs(FFT)",
    range=[-0.1,1])
fig.update_xaxes(
    title_text="frequency (Hz)",)

We call the lowest peak in the spectrum the fundamental frequency. Note that there are other high frequency peaks that follow the fundamental, at regular intervals. These are called overtones, and they are multiples of the fundamental frequency. The fundamental and the overtones are called together the harmonics.

![](crowhurst_basic_audio_vol1-23.jpg)

::: {.column-margin}
Image source: [N.H. Crowhurst](http://www.vias.org/crowhurstba/crowhurst_basic_audio_vol1_009.html)
:::

Probably, the most common tuning standard in western music is A440, meaning that the note corresponding to the A4 on the piano must have a fundamental frequency equal to 440 Hz. Zoom in and find out if the piano was "properly" tuned.

The A3 note has its fundamental frequency at half of that, namely 220 Hz. 

::: {.callout-note collapse="true" icon=false}
## {{< iconify fa music >}} Because there are 12 half-steps between A3 and A4, can you figure out a rule how to find the frequency of any note on the piano?

Dividing an octave in 12 equal half-steps is called "equal temperament". Multiplying the A3 frequency 12 times by an unknown factor $y$ should give us the frequency of A4:

$$
\begin{split}
220 \times y^{12} &= 440\\
y^{12} &= 2 \\
\therefore y &= \sqrt[12]{2}
\end{split}
$$
:::

We can now easily check out if the fundamental frequency for B3 as shown in the graph agrees with this rule:

In [7]:
#| code-summary: "calculate frequency for B4"
factor = 2**(1/12)
B4 = 440 * factor**2  # two half steps above A4
print(f"B4 = {B4:.2f} Hz")

B4 = 493.88 Hz
